# 5DEEP - Projet final : Classification des chants d'oiseaux

Pour rappel:

Il s'agit d'un problème de classification de chants d'oiseaux avec 5 classes, une pour chacune des espèces suivantes : 

- Bewick's Wren
- Northern Cardinal
- American Robin
- Song Sparrow
- Northern Mockingbird

Pour répondre à cette problématique, nous utiliserons des réseaux de neuronnes que nous préparerons et entraînerons grâce à Keras et Librosa dans ce notebook Jupyter.

## Etape 1: Analyse exploratoire et comprehension des données

Nous allons tout d'abord commencer par comprendre les différentes données mises à disposition, mais aussi comprendre l'interpretation de ces fichiers audio et leur traitement.

In [ ]:
import pandas as pd

bird_df = pd.read_csv("data/bird_songs_metadata.csv")

bird_df.head(10)

bird_df.info()
bird_df.isnull().sum()

print("Nombre d'observations:", len(bird_df))

# On retire 'subspecies' car trop corrélée avec 'species' et pas de sens pour notre classification
# 'recordist', 'license', 'remarks', 'source_url' n'apporte pas de sens pour notre classificiation

# On conserve le 'filename' pour retrouver l'enregistrement associé


# est-ce qu'on garde 'sound_type' ?? On filtre ceux qui ne sont pas 'song'

# Notre variable cilble est 'species'

bird_df_song_only = bird_df[bird_df['sound_type'].str.contains("song", case=False, na=False)]

cols = ["id", "genus", "species", "name", "country", "location", 
        "latitude", "longitude", "altitude", "time", "date", "filename", 'sound_type']
bird_df_useful = bird_df_song_only[cols]

bird_df_useful.head(10)

In [ ]:
## Distribution & corrélations

import matplotlib.pyplot as plt

bird_df_useful['species'].value_counts().plot(kind='bar', figsize=(8,4))
plt.title("Nombre d'échantillons par espèce")
plt.xlabel("Espèce")
plt.ylabel("Nombre d'enregistrements")
plt.show()


In [ ]:
## Test librosa

import librosa
import librosa.display
import numpy as np

sample_file = bird_df_useful['filename'].iloc[0]
path = f"data/wavfiles/{sample_file}"
y, sr = librosa.load(path, sr=22050, mono=True)

# forme du signal
plt.figure(figsize=(10,3))
librosa.display.waveshow(y, sr=sr)
plt.title(f"Waveform - {sample_file}")
plt.show()

# mel spectrogram
def extract_mel_spectrogram(file_path, sr=22050, n_mels=128, n_fft=2048, hop_length=512):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    log_mel = librosa.power_to_db(mel, ref=np.max)
    return log_mel.T # shape (time, n_mels)

# mfcc 
def extract_mfcc(file_path, sr=22050, n_mfcc=40, n_fft=2048, hop_length=512):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return mfcc.T  # shape (time, n_mfcc)


# test de la fonction mel spectrogram sur le sample
mel_spec = extract_mel_spectrogram(f"data/wavfiles/{sample_file}")
plt.figure(figsize=(10,4))
librosa.display.specshow(mel_spec.T, sr=sr, hop_length=512,
                            x_axis="time", y_axis="mel")
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Specrtrogram")
plt.show()

# test de la fonction mfcc sur le sample
mfcc_feat = extract_mfcc(f"data/wavfiles/{sample_file}")
plt.figure(figsize=(10,4))
librosa.display.specshow(mfcc_feat.T, sr=sr, hop_length=512,
                            x_axis="time")
plt.colorbar()
plt.title("MFCC")
plt.show()


In [ ]:
## Encoding et pré-traitement des données audio

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# reucp la durée des enregistrements
durations = []
for f in bird_df_useful['filename']:
    y, sr = librosa.load(f"data/wavfiles/{f}", sr=None)
    durations.append(librosa.get_duration(y=y, sr=sr))
bird_df_useful['duration'] = durations

